In [95]:
import pandas as pd
import dask.dataframe as dd
from IPython.core.display import display, HTML
import requests
from bs4 import BeautifulSoup
import os
from dask.diagnostics import ProgressBar

C:\Users\peteramerkhanian\AppData\Local\Temp\ipykernel_17924\1338892348.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [14]:

# URL of the webpage to scrape
url = 'http://64.111.127.166/origin-destination/'

# Send an HTTP GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the webpage with Beautiful Soup
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print("Failed to retrieve the webpage")


In [30]:
links = soup.find_all('a')

In [42]:
hrefs = [l.get('href') for l in links]
hrefs

['?C=N;O=D',
 '?C=M;O=A',
 '?C=S;O=A',
 '?C=D;O=A',
 '/',
 'READ%20ME.txt',
 'date-hour-soo-dest-2011.csv.gz',
 'date-hour-soo-dest-2012.csv.gz',
 'date-hour-soo-dest-2013.csv.gz',
 'date-hour-soo-dest-2014.csv.gz',
 'date-hour-soo-dest-2015.csv.gz',
 'date-hour-soo-dest-2016.csv.gz',
 'date-hour-soo-dest-2017.csv.gz',
 'date-hour-soo-dest-2018.csv.gz',
 'date-hour-soo-dest-2019.csv.gz',
 'date-hour-soo-dest-2020.csv.gz',
 'date-hour-soo-dest-2021.csv.gz',
 'date-hour-soo-dest-2022.csv.gz',
 'date-hour-soo-dest-2023.csv.gz']

In [72]:
files = [f for f in hrefs if '.csv.gz' in f]
files

['date-hour-soo-dest-2011.csv.gz',
 'date-hour-soo-dest-2012.csv.gz',
 'date-hour-soo-dest-2013.csv.gz',
 'date-hour-soo-dest-2014.csv.gz',
 'date-hour-soo-dest-2015.csv.gz',
 'date-hour-soo-dest-2016.csv.gz',
 'date-hour-soo-dest-2017.csv.gz',
 'date-hour-soo-dest-2018.csv.gz',
 'date-hour-soo-dest-2019.csv.gz',
 'date-hour-soo-dest-2020.csv.gz',
 'date-hour-soo-dest-2021.csv.gz',
 'date-hour-soo-dest-2022.csv.gz',
 'date-hour-soo-dest-2023.csv.gz']

In [73]:
file_urls = [url + "/" + f for f in files]
file_urls

['http://64.111.127.166/origin-destination//date-hour-soo-dest-2011.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2012.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2013.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2014.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2015.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2016.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2017.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2018.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2019.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2020.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2021.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2022.csv.gz',
 'http://64.111.127.166/origin-destination//date-hour-soo-dest-2023.csv.gz']

In [60]:
counter = 1
total = 0
for f in file_urls:
    response = requests.head(f)
    file_size = int(response.headers.get('Content-Length', 0))
    total += file_size
    print(f"File {counter} size: {file_size} bytes")
    counter += 1

File 1 size: 35763624 bytes
File 2 size: 37086081 bytes
File 3 size: 36504603 bytes
File 4 size: 37607682 bytes
File 5 size: 38263683 bytes
File 6 size: 37896643 bytes
File 7 size: 37830997 bytes
File 8 size: 38627139 bytes
File 9 size: 38177159 bytes
File 10 size: 21415653 bytes
File 11 size: 24350926 bytes
File 12 size: 30546036 bytes
File 13 size: 27103436 bytes


In [61]:
print(f"Total size of data: {total*10e-7} mega-bytes")

Total size of data: 441.173662 mega-bytes


In [96]:
# Create the "data" folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

In [97]:
from tqdm import tqdm

In [99]:
# Download and save the files
for url in tqdm(file_urls):
    filename = os.path.join('data', os.path.basename(url))
    if os.path.exists(filename):
        print(f"File already exists: {filename}")
    else:
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)
        else:
            print(f"Failed to download: {url}")

100%|██████████| 13/13 [01:53<00:00,  8.75s/it]


In [100]:
# Create a .gitignore file
gitignore_content = "data/\n"  # Content to exclude the "data" folder

with open('.gitignore', 'w') as gitignore_file:
    gitignore_file.write(gitignore_content)

In [104]:
data_paths = ["data/" + f for f in files]
data_paths

['data/date-hour-soo-dest-2011.csv.gz',
 'data/date-hour-soo-dest-2012.csv.gz',
 'data/date-hour-soo-dest-2013.csv.gz',
 'data/date-hour-soo-dest-2014.csv.gz',
 'data/date-hour-soo-dest-2015.csv.gz',
 'data/date-hour-soo-dest-2016.csv.gz',
 'data/date-hour-soo-dest-2017.csv.gz',
 'data/date-hour-soo-dest-2018.csv.gz',
 'data/date-hour-soo-dest-2019.csv.gz',
 'data/date-hour-soo-dest-2020.csv.gz',
 'data/date-hour-soo-dest-2021.csv.gz',
 'data/date-hour-soo-dest-2022.csv.gz',
 'data/date-hour-soo-dest-2023.csv.gz']

In [105]:
df = dd.read_csv(data_paths, blocksize=None)

In [106]:
df.columns = ['Date', 'Hour', 'Start', 'End', 'Riders']

In [107]:
df

,Date,Hour,Start,End,Riders
npartitions=13,,,,,
,object,int64,object,object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [109]:
ProgressBar().register()

In [110]:
df.shape[0].compute()

[########################################] | 100% Completed |  1min 26.6s
[########################################] | 100% Completed |  1min 26.7s


117029110

In [111]:
df.head()

[########################################] | 100% Completed |  4.6s
[########################################] | 100% Completed |  4.7s


,Date,Hour,Start,End,Riders
0,2011-01-01,0,12TH,16TH,1
1,2011-01-01,0,12TH,24TH,3
2,2011-01-01,0,12TH,ASHB,2
3,2011-01-01,0,12TH,BAYF,5
4,2011-01-01,0,12TH,CIVC,3


In [112]:
df_rph = df.groupby(['Date', 'Hour'])['Riders'].sum().compute()

[#########################               ] | 62% Completed |  1min 41.1s